In [ ]:
import json, os
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime


In [ ]:
todo_items = []


In [ ]:
title_input = widgets.Text(placeholder='할 일 입력', layout=widgets.Layout(width='100%'))
progress_input = widgets.IntSlider(min=0, max=100, step=1, value=0, layout=widgets.Layout(width='100%'))
due_input = widgets.DatePicker(placeholder='마감일 선택')
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=widgets.Layout(width='100%'))

title_label = widgets.HTML(value="<b>할 일</b>")
progress_label = widgets.HTML(value="<b>진행률</b>")
due_label = widgets.HTML(value="<b>마감일</b>")
priority_label = widgets.HTML(value="<b>중요도</b>")

add_btn = widgets.Button(description='추가', button_style='success', layout=widgets.Layout(width='100%'))
save_btn = widgets.Button(description='저장', button_style='info', layout=widgets.Layout(width='49%'))
load_btn = widgets.Button(description='불러오기', button_style='warning', layout=widgets.Layout(width='49%'))

progress_bar = widgets.FloatProgress(min=0, max=100, value=0, bar_style='', layout=widgets.Layout(width='100%'))
progress_text = widgets.Label()

list_box = widgets.VBox()


In [ ]:
def update_overall_progress():
    if not todo_items:
        progress_bar.value = 0
        progress_text.value = ""
        return
    avg = sum([item['progress'] for item in todo_items]) / len(todo_items)
    progress_bar.value = avg
    progress_text.value = f"{int(avg)}%"

def refresh_list():
    children = []
    for i, item in enumerate(todo_items, 1):
        title = widgets.Label(value=f"# {i}. {item['title']}", layout=widgets.Layout(width='100%'))
        bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=widgets.Layout(width='100%'), bar_style='')
        slider = widgets.IntSlider(value=item['progress'], min=0, max=100, layout=widgets.Layout(width='100%'))

        def update_progress(change, idx=i-1):
            todo_items[idx]['progress'] = change['new']
            refresh_list()
            update_overall_progress()

        slider.observe(update_progress, names='value')

        meta = widgets.HBox([
            widgets.Label(value=f"마감일 {item['due']}"),
            widgets.Label(value=f"중요도 {item['priority']}")
        ], layout=widgets.Layout(justify_content='space-between'))

        done_btn = widgets.Button(description='완료✅', layout=widgets.Layout(width='49%'))
        del_btn = widgets.Button(description='삭제❌', layout=widgets.Layout(width='49%'))

        def mark_done(btn, idx=i-1):
            todo_items[idx]['progress'] = 100
            refresh_list()
            update_overall_progress()

        def delete_task(btn, idx=i-1):
            del todo_items[idx]
            refresh_list()
            update_overall_progress()

        done_btn.on_click(mark_done)
        del_btn.on_click(delete_task)

        box = widgets.VBox([title, bar, slider, meta, widgets.HBox([done_btn, del_btn])],
                           layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='10px 0px'))
        children.append(box)

    list_box.children = children


In [ ]:
def add_task(btn):
    if not title_input.value.strip():
        return
    todo_items.append({
        'title': title_input.value.strip(),
        'progress': progress_input.value,
        'due': due_input.value.strftime('%Y-%m-%d') if due_input.value else '',
        'priority': priority_input.value
    })
    title_input.value = ''
    progress_input.value = 0
    due_input.value = None
    priority_input.value = '보통'
    refresh_list()
    update_overall_progress()

def save_data(btn):
    with open('todo.json', 'w') as f:
        json.dump(todo_items, f)

def load_data(btn):
    if not os.path.exists('todo.json'):
        return
    with open('todo.json', 'r') as f:
        data = json.load(f)
        todo_items.clear()
        todo_items.extend(data)
        refresh_list()
        update_overall_progress()

add_btn.on_click(add_task)
save_btn.on_click(save_data)
load_btn.on_click(load_data)


In [ ]:
display(widgets.HTML(value="<h3 style='color:black;'>📁 DAVID mk40 - 새 시작</h3>"))
display(widgets.VBox([
    widgets.HBox([title_label, title_input]),
    widgets.HBox([progress_label, progress_input]),
    widgets.HBox([due_label, due_input, priority_label, priority_input]),
    add_btn,
    widgets.HBox([save_btn, load_btn]),
    widgets.HTML(value="<b>전체 진행률</b>"),
    widgets.HBox([progress_bar, progress_text]),
    list_box
]))
refresh_list()
update_overall_progress()
